# Лабораторная работа №:6 Классификация текста
**Выполнил:** Бибиков Павел, ИУ5-22М

## Цель лабораторной работы: 
Изучение методов классификации текстов.

## Задание:
Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:

1. Способ 1. На основе CountVectorizer или TfidfVectorizer.
2. Способ 2. На основе моделей word2vec или Glove или fastText.
3. Сравните качество полученных моделей.

Для поиска наборов данных в поисковой системе можно использовать ключевые слова "datasets for text classification".

In [28]:
import pandas as pd
import numpy as np
import re
import nltk # Natural Language Toolkit

# !!! БЛОК ПРОВЕРКИ И ЗАГРУЗКИ NLTK РЕСУРСОВ !!!
# !!! ЗАПУСТИТЕ ЭТУ ЯЧЕЙКУ, ДОЖДИТЕСЬ ЗАВЕРШЕНИЯ, ЗАТЕМ ПЕРЕЗАПУСТИТЕ ЯДРО !!!
# !!! (Kernel -> Restart) ПЕРЕД ЗАПУСКОМ ОСТАЛЬНЫХ ЯЧЕЕК !!!

print("Проверка и загрузка NLTK ресурсов...")
# Список ресурсов и их ожидаемые пути (частичные) для nltk.data.find()
resources_to_download = {
    'punkt': 'tokenizers/punkt',
    'stopwords': 'corpora/stopwords',
    'wordnet': 'corpora/wordnet.zip', # Проверяем наличие zip-файла или основного каталога
    'omw-1.4': 'corpora/omw-1.4.zip'  # Аналогично
}
# Идентификаторы для nltk.download() (могут совпадать или немного отличаться от ключей выше)
download_ids = {
    'punkt': 'punkt',
    'stopwords': 'stopwords',
    'wordnet': 'wordnet',
    'omw-1.4': 'omw-1.4'
}


all_resources_found_and_loaded = True
for resource_name, resource_path_check in resources_to_download.items():
    download_id = download_ids[resource_name]
    try:
        # Сначала пытаемся найти ресурс
        nltk.data.find(resource_path_check)
        print(f"- Ресурс '{resource_name}' найден (проверка по пути: {resource_path_check}).")
    except LookupError:
        print(f"- Ресурс '{resource_name}' НЕ найден (проверка по пути: {resource_path_check}). Попытка загрузки ID: '{download_id}'...")
        try:
            nltk.download(download_id, quiet=False) # quiet=False для подробного вывода
            # Повторная проверка после явной загрузки
            nltk.data.find(resource_path_check)
            print(f"- Ресурс '{resource_name}' успешно загружен и найден.")
        except Exception as e_download:
            print(f"  ** Ошибка при загрузке или повторной проверке '{resource_name}' (ID: '{download_id}'): {e_download} **")
            all_resources_found_and_loaded = False
    except Exception as e_find: # Другие возможные ошибки при nltk.data.find
        print(f"  ** Ошибка при поиске '{resource_name}': {e_find} **")
        all_resources_found_and_loaded = False


if all_resources_found_and_loaded:
    print("\nВсе основные NLTK ресурсы успешно проверены/загружены.")
else:
    print("\nНе все NLTK ресурсы удалось корректно загрузить или проверить. Пожалуйста, проверьте вывод выше и ваше интернет-соединение.")
    print("Убедитесь, что директория nltk_data (обычно C:\\Users\\YourUser\\AppData\\Roaming\\nltk_data) доступна для записи.")

# !!! КОНЕЦ БЛОКА ПРОВЕРКИ/ЗАГРУЗКИ NLTK !!!


# Остальные импорты
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import gensim
import gensim.downloader as api

import matplotlib.pyplot as plt
import seaborn as sns

# Настройки
pd.set_option('display.max_colwidth', 200)

Проверка и загрузка NLTK ресурсов...
- Ресурс 'punkt' найден (проверка по пути: tokenizers/punkt).
- Ресурс 'stopwords' найден (проверка по пути: corpora/stopwords).
- Ресурс 'wordnet' найден (проверка по пути: corpora/wordnet.zip).
- Ресурс 'omw-1.4' найден (проверка по пути: corpora/omw-1.4.zip).

Все основные NLTK ресурсы успешно проверены/загружены.


In [29]:
# Загрузка данных SMS Spam Collection

# ИСПОЛЬЗУЙТЕ ЭТОТ БЛОК ДЛЯ ЛОКАЛЬНОГО ФАЙЛА
file_path = 'SMSSpamCollection' # Убедитесь, что файл SMSSpamCollection находится в той же директории
try:
    df = pd.read_csv(file_path, sep='\t', header=None, names=['label', 'message'], encoding='latin-1') # Добавил encoding
    print(f"Файл '{file_path}' успешно загружен.")
except FileNotFoundError:
    print(f"Файл '{file_path}' не найден. Убедитесь, что он находится в нужной директории или укажите правильный путь.")
    # Создадим пустой DataFrame, чтобы код не падал, но анализ будет невозможен
    df = pd.DataFrame(columns=['label', 'message'])
except Exception as e:
    print(f"Произошла ошибка при чтении файла: {e}")
    df = pd.DataFrame(columns=['label', 'message'])


print("Первые 5 строк данных:")
print(df.head())

print("\nИнформация о датасете:")
df.info()

print("\nРаспределение классов:")
if not df.empty: # Проверка, что df не пуст
    print(df['label'].value_counts())
    # Преобразуем метки классов в числовой формат (0 для 'ham', 1 для 'spam')
    df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})
    print("\nДанные с числовыми метками:")
    print(df.head())
else:
    print("DataFrame пуст, информация о классах и числовые метки не могут быть созданы.")

Файл 'SMSSpamCollection' успешно загружен.
Первые 5 строк данных:
  label  \
0   ham   
1   ham   
2  spam   
3   ham   
4   ham   

                                                                                                                                                       message  
0                                              Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...  
1                                                                                                                                Ok lar... Joking wif u oni...  
2  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's  
3                                                                                                            U dun say so early hor... U c already then say...  
4                                                             

In [30]:
# Предобработка текста
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Удаление специальных символов и цифр (оставляем только буквы и пробелы)
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    text = text.lower() # Приведение к нижнему регистру
    tokens = word_tokenize(text) # Токенизация
    # Удаление стоп-слов и лемматизация
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 1]
    return " ".join(lemmatized_tokens)

# Применяем предобработку к сообщениям
# Проверим, есть ли данные в df, прежде чем применять обработку
if not df.empty and 'message' in df.columns:
    df['processed_message'] = df['message'].apply(preprocess_text)
    print("\nДанные после предобработки текста (первые 5 строк):")
    print(df[['message', 'processed_message']].head())
else:
    print("\nDataFrame пуст или отсутствует колонка 'message'. Предобработка не выполнена.")
    df['processed_message'] = "" # Создадим пустую колонку, чтобы код дальше не падал

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\mesou/nltk_data'
    - 'c:\\studies\\MMO\\.venv\\nltk_data'
    - 'c:\\studies\\MMO\\.venv\\share\\nltk_data'
    - 'c:\\studies\\MMO\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\mesou\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
